# NLP | Modelo de Question Answering em qualquer idioma baseado no BERT base (estudo de caso em português)

- **Credit**: this notebook is a modified version of the notebook [question_answering.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb) of Hugging Face. Each new cell is signalized by **new** and each modificated one by **modified**.
- **Acknowledgement**: Neuralmind.ai, Hugging Face, Deep Learning Brasil group, Google Colab and AI Lab ([see the whole list](https://medium.com/@pierre_guillou/nlp-modelo-de-question-answering-em-qualquer-idioma-baseado-no-bert-base-estudo-de-caso-em-12093d385e78#c572)).
- **Author**: [Pierre Guillou](https://www.linkedin.com/in/pierreguillou/)
- **Date**: 12/02/2021
- **Blog post**: [NLP | Modelo de Question Answering em qualquer idioma baseado no BERT base (estudo de caso em português)](https://medium.com/@pierre_guillou/nlp-modelo-de-question-answering-em-qualquer-idioma-baseado-no-bert-base-estudo-de-caso-em-12093d385e78)
- **Model in the Model Hub of Hugging Face**: [Portuguese BERT base cased QA (Question Answering), finetuned on SQUAD v1.1](https://huggingface.co/pierreguillou/bert-base-cased-squad-v1.1-portuguese)

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers

     |████████████████████████████████| 225kB 32.3MB/s 
     |████████████████████████████████| 2.3MB 37.2MB/s 
     |████████████████████████████████| 245kB 51.6MB/s 
     |████████████████████████████████| 112kB 59.0MB/s 
     |████████████████████████████████| 3.3MB 37.4MB/s 
     |████████████████████████████████| 901kB 33.4MB/s 


In [2]:
import sys; print('python:',sys.version)

import transformers; print('transformers:',transformers.__version__)

import torch; print('Pytorch:',torch.__version__)

python: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
transformers: 4.6.0
Pytorch: 1.8.1+cu101


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [4]:
!nvidia-smi

Tue May 18 22:44:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/question-answering).

# Fine-tuning a model on a question-answering task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context. We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

![Widget inference representing the QA task](https://github.com/huggingface/notebooks/blob/master/examples/images/question_answering.png?raw=1)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [5]:
# modified 

# # This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# # answers are allowed or not).
# squad_v2 = False
# model_checkpoint = "distilbert-base-uncased"
# batch_size = 16

In [6]:
# new

# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [7]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [SQUAD dataset](https://rajpurkar.github.io/SQuAD-explorer/). The notebook should work with any question answering dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [8]:
# modified 

# datasets = load_dataset("squad_v2" if squad_v2 else "squad")

In [9]:
# new

# Get dataset SQUAD in Portuguese
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn" -O squad-pt.tar.gz && rm -rf /tmp/cookies.txt

--2021-05-18 22:44:39--  https://docs.google.com/uc?export=download&confirm=qyCp&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn
Resolving docs.google.com (docs.google.com)... 142.250.65.78, 2607:f8b0:4004:832::200e
Connecting to docs.google.com (docs.google.com)|142.250.65.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-9g-docs.googleusercontent.com/docs/securesc/e6clhubq1pgp61ljvbmg86gcut8vjbh3/sm89m9527ok6600nbdrc8f5b9evrd189/1621377825000/03445611175480770093/06948006921668716601Z/1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn?e=download [following]
--2021-05-18 22:44:39--  https://doc-0k-9g-docs.googleusercontent.com/docs/securesc/e6clhubq1pgp61ljvbmg86gcut8vjbh3/sm89m9527ok6600nbdrc8f5b9evrd189/1621377825000/03445611175480770093/06948006921668716601Z/1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn?e=download
Resolving doc-0k-9g-docs.googleusercontent.com (doc-0k-9g-docs.googleusercontent.com)... 142.251.33.193, 2607:f8b0:4004:837::2001
Connecting to doc-0k-

In [10]:
# new

!tar -xvf squad-pt.tar.gz

squad-train-v1.1.json
squad-dev-v1.1.json


In [11]:
%%time
# new

# Get the train and validation json file in the HF script format 
# inspiration: file squad.py at https://github.com/huggingface/datasets/tree/master/datasets/squad

import json 

files = ['squad-train-v1.1.json','squad-dev-v1.1.json']

for file in files:
    
    # Opening JSON file & returns JSON object as a dictionary 
    f = open(file, encoding="utf-8") 
    data = json.load(f) 
    
    # Iterating through the json list 
    entry_list = list()
    id_list = list()

    for row in data['data']: 
        title = row['title']
        
        for paragraph in row['paragraphs']:
            context = paragraph['context']

            for qa in paragraph['qas']:
                entry = {}

                qa_id = qa['id']
                question = qa['question']
                answers = qa['answers']
                
                entry['id'] = qa_id
                entry['title'] = title.strip()
                entry['context'] = context.strip()
                entry['question'] = question.strip()
                
                answer_starts = [answer["answer_start"] for answer in answers]
                answer_texts = [answer["text"].strip() for answer in answers]
                entry['answers'] = {}
                entry['answers']['answer_start'] = answer_starts
                entry['answers']['text'] = answer_texts

                entry_list.append(entry)
                
    reverse_entry_list = entry_list[::-1]
    
    # for entries with same id, keep only last one (corrected texts by he group Deep Learning Brasil)
    unique_ids_list = list()
    unique_entry_list = list()
    for entry in reverse_entry_list:
        qa_id = entry['id']
        if qa_id not in unique_ids_list:
            unique_ids_list.append(qa_id)
            unique_entry_list.append(entry)
        
    # Closing file 
    f.close() 

    new_dict = {}
    new_dict['data'] = unique_entry_list

    file_name = 'pt_' + str(file)
    with open(file_name, 'w') as json_file:
        json.dump(new_dict, json_file)

CPU times: user 1min 59s, sys: 670 ms, total: 1min 59s
Wall time: 1min 59s


In [12]:
!ls

pt_squad-dev-v1.1.json	  sample_data	       squad-pt.tar.gz
pt_squad-train-v1.1.json  squad-dev-v1.1.json  squad-train-v1.1.json


In [13]:
# new

from datasets import load_dataset, load_metric
datasets = load_dataset('json', 
                        data_files={'train': 'pt_squad-train-v1.1.json', 'validation': 'pt_squad-dev-v1.1.json'}, 
                        field='data')

Using custom data configuration default-7334ad1bc94a65ad


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-7334ad1bc94a65ad/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87510
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

We can see the training, validation and test sets all have a column for the context, the question and the answers to those questions.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
# modified

# datasets["train"][0]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [ ]:
# new

datasets["train"][0]

{'answers': {'answer_start': [2],
  'text': ['Cidade Metropolitana de Catmandu']},
 'context': 'A Cidade Metropolitana de Catmandu (KMC), a fim de promover as relações internacionais, criou uma Secretaria de Relações Internacionais (IRC). O primeiro relacionamento internacional da KMC foi estabelecido em 1975 com a cidade de Eugene, Oregon, Estados Unidos. Essa atividade foi aprimorada ainda mais com o estabelecimento de relações formais com outras 8 cidades: Cidade de Motsumoto, Japão, Rochester, EUA, Yangon (antiga Rangum) de Mianmar, Xian da República Popular da China, Minsk da Bielorrússia e Pyongyang de República Democrática da Coréia. O esforço constante da KMC é aprimorar sua interação com os países da SAARC, outras agências internacionais e muitas outras grandes cidades do mundo para alcançar melhores programas de gestão urbana e desenvolvimento para Katmandu.',
 'id': '5735d259012e2f140011a0a1',
 'question': 'De que KMC é um inicialismo?',
 'title': 'Kathmandu'}

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,56de3e40cffd8e1900b4b6dc,Separation_of_powers_under_the_United_States_Constitution,"O presidente exerce um controle sobre o Congresso através de seu poder de vetar projetos de lei, mas o Congresso pode substituir qualquer veto (excluindo o chamado ""veto de bolso"") por maioria de dois terços em cada casa. Quando as duas casas do Congresso não chegarem a um prazo de adiamento, o presidente poderá resolver a disputa. Qualquer casa ou ambas as casas podem ser convocadas para uma sessão de emergência pelo presidente. O vice-presidente atua como presidente do Senado, mas ele só pode votar para romper o empate.",Quem pode determinar uma data de adiamento se o congresso não concordar?,"{'answer_start': [422], 'text': ['presidente']}"
1,572f871db2c2fd14005681c0,Armenia,"Durante o final do século VI aC, a primeira entidade geográfica chamada Armênia pelas populações vizinhas foi estabelecida sob a dinastia Orontid, dentro do Império Aquemênida, como parte dos territórios dos últimos. O reino tornou-se totalmente soberano da esfera de influência do Império Selêucida em 190 aC, sob o rei Artaxias I, e iniciou o governo da dinastia Artaxiad. A Armênia atingiu seu auge entre 95 e 66 aC sob Tigranes, o Grande, tornando-se o reino mais poderoso de seu tempo a leste da República Romana.",Qual governante é responsável pela prosperidade de Armênia?,"{'answer_start': [423], 'text': ['Tigranes, o Grande']}"
2,5707feb09e06ca38007e94d9,Imamah_(Shia_doctrine),"Os xiitas acreditam que Imamah é dos Princípios da Fé (Usul al-Din). Como o versículo 4: 165 do Alcorão expressa a necessidade da nomeação dos profetas; assim, após a morte do profeta, que desempenhará o papel do profeta; até que o povo não tenha nenhum argumento contra Allah. Portanto, a mesma lógica que exigia a designação de profetas também é aplicada a Imama. Isso é Allah Deve designar alguém semelhante ao profeta em seus atributos e Ismah como seu sucessor para guiar o povo sem nenhum desvio. na religião. Eles se referem ao versículo (... Neste dia eu aperfeiçoei para você sua religião e completei Meu favor sobre você e aprovei para você o Islã como religião ...) 5: 3 do Alcorão que foi revelado ao profeta quando ele designou Ali como seu sucessor no dia de Ghadir Khumm.",Quem acredita que Imamah é dos Princípios da Fé?,"{'answer_start': [3], 'text': ['xiitas']}"
3,56f8fba19b226e1400dd1231,Near_East,"Posteriormente, com a desgraça do ""Próximo Oriente"" nos círculos diplomáticos e militares, o ""Oriente Médio"" prevaleceu. No entanto, o ""Oriente Próximo"" continua em alguns círculos, a critério da agência ou departamento acadêmico de definição. Eles geralmente não são considerados regiões distintas, como estavam em sua definição original.","O que continua em alguns círculos, a critério da agência ou departamento acadêmico definidor?","{'answer_start': [136], 'text': ['Oriente Próximo']}"
4,56f72a10711bf01900a44a16,Classical_music,"Embora o temperamento igual tenha sido gradualmente aceito como o temperamento musical dominante durante o século 18, diferentes temperamentos históricos são frequentemente usados para a música de períodos anteriores. Por exemplo, a música do Renascimento inglês é frequentemente tocada em temperamento intencional.",Que tipo de temperamento foi aceito no século XVIII?,"{'answer_start': [9], 'text': ['temperamento igual']}"
5,56fb7c108ddada1400cd6458,Middle_Ages,"No centro e no norte da Itália e na Flandres, a ascensão de cidades autônomas estimulou o crescimento econômico e criou um ambiente para novos tipos de associações comerciais. As cidades comerciais às margens do Báltico entraram em acordos conhecidos como Liga Hanseática, e as repúblicas marítimas italianas como Veneza, Génova e Pisa expandiram seu comércio por todo o Mediterrâneo. [V] Grandes feiras comerciais foram estabelecidas e floresceram no norte da França. durante o período, permitindo que comerciantes italianos e alemães negociassem entre si, bem como comerc

## Preprocessing the training data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on two sentences (one for the answer, one for the context):

In [ ]:
# new

# tokenizer("What is your name?", "My name is Sylvain.")
tokenizer("Como se chama?", "Eu me chamo Sylvain.")

{'input_ids': [101, 1602, 176, 3196, 136, 102, 3396, 311, 849, 22280, 16775, 256, 147, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [ ]:
# new

example

{'answers': {'answer_start': [512], 'text': ['Junho a agosto']},
 'context': 'A cidade geralmente tem um clima com dias quentes, seguido de noites e manhãs frias. Prevê-se um clima imprevisível, uma vez que as temperaturas podem cair para 1 ° C (34 ° F) ou menos durante o inverno. Durante uma frente fria de 2013, as temperaturas de inverno de Katmandu caíram para -4 ° C (25 ° F), e a temperatura mais baixa foi registrada em 10 de janeiro de 2013, a -9,2 ° C (15,4 ° F). As chuvas são principalmente baseadas em monções (cerca de 65% do total concentrado durante os meses das monções de Junho a agosto) e diminuem substancialmente (100 a 200 cm) do leste do Nepal ao oeste do Nepal. As chuvas foram registradas em cerca de 1.400 milímetros (55,1 polegadas) no vale de Katmandu e em média 1.407 milímetros (55,4 polegadas) na cidade de Katmandu. Em média, a umidade é de 75%. O gráfico abaixo é baseado em dados do Bureau de Padrões e Meteorologia do Nepal, "Meteorologia Meteorológica" de 2005. O 

Without any truncation, we get the following length for the input IDs:

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

407

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

Note that we never want to truncate the question, only the context, else the `only_second` truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with `return_overflowing_tokens=True` and by passing the stride:

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Now we don't have one list of `input_ids`, but several: 

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 165]

And if we decode them, we can see the overlap:

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] Em que meses a monção ocorre no Nepal? [SEP] A cidade geralmente tem um clima com dias quentes, seguido de noites e manhãs frias. Prevê - se um clima imprevisível, uma vez que as temperaturas podem cair para 1 ° C ( 34 ° F ) ou menos durante o inverno. Durante uma frente fria de 2013, as temperaturas de inverno de Katmandu caíram para - 4 ° C ( 25 ° F ), e a temperatura mais baixa foi registrada em 10 de janeiro de 2013, a - 9, 2 ° C ( 15, 4 ° F ). As chuvas são principalmente baseadas em monções ( cerca de 65 % do total concentrado durante os meses das monções de Junho a agosto ) e diminuem substancialmente ( 100 a 200 cm ) do leste do Nepal ao oeste do Nepal. As chuvas foram registradas em cerca de 1. 400 milímetros ( 55, 1 polegadas ) no vale de Katmandu e em média 1. 407 milímetros ( 55, 4 polegadas ) na cidade de Katmandu. Em média, a umidade é de 75 %. O gráfico abaixo é baseado em dados do Bureau de Padrões e Meteorologia do Nepal, " Meteorologia Meteorológica " de 2005. O

Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an `offset_mapping`:

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 2), (3, 6), (7, 12), (13, 14), (15, 18), (18, 21), (22, 28), (29, 31), (32, 34), (34, 37), (37, 38), (0, 0), (0, 1), (2, 8), (9, 19), (20, 23), (24, 26), (27, 32), (33, 36), (37, 41), (42, 49), (49, 50), (51, 58), (59, 61), (62, 68), (69, 70), (71, 76), (76, 77), (78, 82), (82, 83), (83, 84), (85, 88), (88, 90), (90, 91), (91, 93), (94, 96), (97, 102), (103, 108), (108, 115), (115, 116), (117, 120), (121, 124), (125, 128), (129, 131), (132, 144), (145, 150), (151, 155), (156, 160), (161, 162), (163, 164), (165, 166), (167, 168), (168, 170), (171, 172), (173, 174), (174, 175), (176, 178), (179, 184), (185, 192), (193, 194), (195, 202), (202, 203), (204, 211), (212, 215), (216, 222), (223, 227), (228, 230), (231, 235), (235, 236), (237, 239), (240, 252), (253, 255), (256, 263), (264, 266), (267, 270), (270, 274), (274, 275), (276, 282), (283, 287), (288, 289), (289, 290), (291, 292), (293, 294), (295, 296), (296, 298), (299, 300), (301, 302), (302, 303), (303, 304), (305, 30

This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token (`[CLS]`) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

Em Em


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the `sequence_ids` method of our `tokenized_example` can be useful:

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns `None` for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

143 145


And we can double check that it is indeed the theoretical answer:

In [ ]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

Junho a agosto
Junho a agosto


For this notebook to work with any kind of models, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context):

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [ ]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
features = prepare_train_features(datasets['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
# new 

file_name = str(model_checkpoint).replace('/','-') + '-finetuned-squad-v1.1-pt'

In [ ]:
# modified 

# args = TrainingArguments(
#     f"test-squad",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

In [ ]:
# new 

args = TrainingArguments(
    f"{file_name}",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3.0,
    weight_decay=0.01,
    save_total_limit=1,
    fp16 = True
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

We will evaluate our model and compute metrics in the next section (this is a very long operation, so we will only compute the evaluation loss during training).

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,1.024800,1.075474,291.106300,37.502000
2,0.756400,1.104146,291.190400,37.491000
3,0.524000,1.225836,291.215800,37.488000


TrainOutput(global_step=16734, training_loss=0.8457311714822507, metrics={'train_runtime': 9533.1723, 'train_samples_per_second': 1.755, 'total_flos': 66822624992549376, 'epoch': 3.0})

Since this training is particularly long, let's save the model just in case we need to restart.

In [ ]:
trainer.save_model(file_name)

## Evaluation

Evaluating our model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and en position of our answers: if we take a batch from our validation datalaoder, here is the output our model gives us:

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

The output of the model is a dict-like object that contains the loss (since we provided labels), the start and end logits. We won't need the loss for our predictions, let's have a look a the logits:

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

We have one logit for each feature and each token. The most obvious thing to predict an answer for each featyre is to take the index for the maximum of the start logits as a start position and the index of the maximum of the end logits as an end position.

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 15,  80, 114,  76, 183,  73,  22,  40,  40, 128,  47,  53,  53,  39,
          39,  97], device='cuda:0'),
 tensor([ 18,  84, 116,  80, 190,  80,  22,  41,  42, 149,  52,  42,  42,  51,
          51, 115], device='cuda:0'))

This will work great in a lot of cases, but what if this prediction gives us something impossible: the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

However, picking the second best answer is not as easy as picking the best one: is it the second best index in the start logits with the best index in the end logits? Or the best index in the start logits with the second best index in the end logits? And if that second best answer is not possible either, it gets even trickier for the third best answer.


To classify our answers, we will use the score obtained by adding the start and end logits. We won't try to order all the possible answers and limit ourselves to with a hyper-parameter we call `n_best_size`. We'll pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we will sort them by their score and keep the best one. Here is how we would do this on the first feature in the batch:

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

And then we can sort the `valid_answers` according to their `score` and only keep the best one. The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [ ]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [ ]:
raw_predictions = trainer.predict(validation_features)

The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing), so we set them back:

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [ ]:
max_answer_length = 30

In [ ]:
# new
num_qa = 0
datasets["validation"][num_qa]

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][num_qa]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

We can compare to the actual ground-truth answer:

In [ ]:
# modified
datasets["validation"][num_qa]["answers"]

Our model picked the right as the most likely answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer (when `squad_v2 = True`). The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10917 features.


Then we can load the metric from the datasets library.

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 70.49195837275307, 'f1': 82.50069751388098}

Don't forget to [update your model](https://huggingface.co/transformers/model_sharing.html) on the [🤗 Model Hub](https://huggingface.co/models). You can then use it only to generate results like the one shown in the first picture of this notebook!|

# Save our QA model to HF format

In [ ]:
import pathlib
from pathlib import Path

In [ ]:
dest = pathlib.Path.cwd()/'HFmodels'
fname_HF = file_name

path_to_awesome_name_you_picked = dest/fname_HF
path_to_awesome_name_you_picked.mkdir(exist_ok=True, parents=True)

In [ ]:
trainer.save_model(f"{str(path_to_awesome_name_you_picked)}")
tokenizer.save_pretrained(f"{str(path_to_awesome_name_you_picked)}")

('/content/HFmodels/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/tokenizer_config.json',
 '/content/HFmodels/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/special_tokens_map.json',
 '/content/HFmodels/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/vocab.txt',
 '/content/HFmodels/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/added_tokens.json')

In [ ]:
# model size
path_file = path_to_awesome_name_you_picked/'pytorch_model.bin'
print('(',int(round(path_file.stat().st_size/(1024*1024),0)),' M)',path_file.parent.name)
        
print('\nmodel saved!')

( 413  M) neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt

model saved!


# Download the model files to your local folder

In [ ]:
# file_to_download = '/content/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/vocab.txt'
# file_to_download = '/content/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/training_args.bin'
# file_to_download = '/content/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/tokenizer_config.json'
# file_to_download = '/content/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/special_tokens_map.json'
# file_to_download = '/content/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/pytorch_model.bin'
file_to_download = '/content/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt/config.json'

In [ ]:
from google.colab import files
files.download(file_to_download)

# Use our QA model

In [ ]:
import transformers

In [ ]:
import pathlib
from pathlib import Path

In [ ]:
dest = pathlib.Path.cwd()/'HFmodels'
fname_HF = 'neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt'

path_to_awesome_name_you_picked = dest/fname_HF

In [ ]:
from transformers import pipeline

In [ ]:
# source: https://pt.wikipedia.org/wiki/Pandemia_de_COVID-19
context = r"""
A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19, 
uma doença respiratória aguda causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2). 
A doença foi identificada pela primeira vez em Wuhan, na província de Hubei, República Popular da China, 
em 1 de dezembro de 2019, mas o primeiro caso foi reportado em 31 de dezembro do mesmo ano. 
Acredita-se que o vírus tenha uma origem zoonótica, porque os primeiros casos confirmados 
tinham principalmente ligações ao Mercado Atacadista de Frutos do Mar de Huanan, que também vendia animais vivos. 
Em 11 de março de 2020, a Organização Mundial da Saúde declarou o surto uma pandemia. Até 8 de fevereiro de 2021, 
pelo menos 105 743 102 casos da doença foram confirmados em pelo menos 191 países e territórios, 
com cerca de 2 308 943 mortes e 58 851 440 pessoas curadas.
"""

In [ ]:
model_qa = str(path_to_awesome_name_you_picked)
nlp = pipeline("question-answering", model=model_qa)

Some weights of BertModel were not initialized from the model checkpoint at /mnt/home/pierre/course-v4/nbs/QA/HFmodels/colab_neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt_12fev2021 and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
question = "Quando começou a pandemia de Covid-19 no mundo?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: '1 de dezembro de 2019', score: 0.713, start: 328, end: 349


In [ ]:
question = "Qual é a data de início da pandemia Covid-19 em todo o mundo?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: '1 de dezembro de 2019', score: 0.4922, start: 328, end: 349


In [ ]:
question = "O qué a pandemia de Covid-19?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'uma doença respiratória aguda', score: 0.0429, start: 110, end: 139


In [ ]:
question = "A Covid-19 tem algo a ver com animais?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'Acredita-se que o vírus tenha uma origem zoonótica', score: 0.3732, start: 418, end: 468


In [ ]:
question = "Onde foi descoberta a Covid-19?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'Wuhan, na província de Hubei, República Popular da China', score: 0.4265, start: 266, end: 322


In [ ]:
question = "Quantos casos houve?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: '105 743 102', score: 0.4607, start: 750, end: 761


In [ ]:
question = "Quantos mortes?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: '2 308 943', score: 0.7313, start: 850, end: 859


In [ ]:
question = "Quantos paises tiveram casos?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: '191', score: 0.4053, start: 810, end: 813


In [ ]:
question = "Quantas pessoas foram curadas?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: '58 851 440', score: 0.9288, start: 869, end: 879


# Upload our QA model to the Model Hub of HF

Source: [update your model](https://huggingface.co/transformers/model_sharing.html) on the [🤗 Model Hub](https://huggingface.co/models). 

In [ ]:
import transformers

In [ ]:
import pathlib
from pathlib import Path

In [ ]:
dest = pathlib.Path.cwd()/'HFmodels'

# new name of 'neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt'
fname_HF = 'bert-base-cased-squad-v1.1-portuguese'

path_to_awesome_name_you_picked = dest/fname_HF

## Save locally 

Done in the paragraph "Save our QA model to HF format" above.

## Upload your model with the CLI

Now go in a terminal and run the following command. It should be in the virtual enviromnent where you installed 🤗 Transformers, since that command transformers-cli comes from the library.

`transformers-cli login`

Then log in using the same credentials as on huggingface.co. 

Once you are logged in with your model hub credentials, you can start building your repositories. To create a repo:
    
`transformers-cli repo create your-model-name --organization your-org-name`

This creates a repo on the model hub, which can be cloned.

```
# Make sure you have git-lfs installed
# (https://git-lfs.github.com/)
git lfs install

git clone https://huggingface.co/username/your-model-name
```

This creates a repo on the model hub, which can be cloned.

```

# Make sure you have git-lfs installed
# (https://git-lfs.github.com/)
git lfs install

git clone https://huggingface.co/username/your-model-name
```

When you have your local clone of your repo and lfs installed, you can then add/remove from that clone as you would with any other git repo.

```

# Commit as usual
cd your-model-name
echo "hello" >> README.md
git add . && git commit -m "Update from $USER"
```

## Make your model work on all frameworks

In [ ]:
# !pip install --upgrade tensorflow
# !pip install --upgrade tensorflow

In [ ]:
from transformers import TFBertForQuestionAnswering
import tensorflow as tf

In [ ]:
tf_model = TFBertForQuestionAnswering.from_pretrained(str(path_to_awesome_name_you_picked), from_pt=True)
tf_model.save_pretrained(str(path_to_awesome_name_you_picked))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForQuestionAnswering: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


## Check the directory before pushing to the model hub.

Make sure there are no garbage files in the directory you’ll upload. It should only have:

- a config.json file, which saves the configuration of your model ;
- a pytorch_model.bin file, which is the PyTorch checkpoint (unless you can’t have it for some reason) ;
- a tf_model.h5 file, which is the TensorFlow checkpoint (unless you can’t have it for some reason) ;
- a special_tokens_map.json, which is part of your tokenizer save;
- a tokenizer_config.json, which is part of your tokenizer save;
- files named vocab.json, vocab.txt, merges.txt, or similar, which contain the vocabulary of your tokenizer, part of your tokenizer save;
- maybe a added_tokens.json, which is part of your tokenizer save.

Other files can safely be deleted.

## Uploading your files

Once the repo is cloned, you can add the model, configuration and tokenizer files. For instance, saving the model and tokenizer files:

```
>>> model.save_pretrained("path/to/repo/clone/your-model-name")
>>> tokenizer.save_pretrained("path/to/repo/clone/your-model-name")
```

Or, if you’re using the Trainer API

```
>>> trainer.save_model("path/to/awesome-name-you-picked")
>>> tokenizer.save_pretrained("path/to/repo/clone/your-model-name")
```

You can then add these files to the staging environment and verify that they have been correctly staged with the git status command:

```
git add --all
git status
```

Finally, the files should be committed:

```
git commit -m "First version of the your-model-name model and tokenizer."
```

And pushed to the remote:

```
git push
```

This will upload the folder containing the weights, tokenizer and configuration we have just prepared.

## Add a model card

To make sure everyone knows what your model can do, what its limitations, potential bias or ethical considerations are, please add a README.md model card to your model repo. You can just create it, or there’s also a convenient button titled “Add a README.md” on your model page. A model card template can be found [here](https://github.com/huggingface/model_card) (meta-suggestions are welcome). model card template (meta-suggestions are welcome).